In [42]:
import os

import cohere
import streamlit as st
from langchain_cohere import CohereEmbeddings
from langchain_pinecone import PineconeVectorStore

os.environ["PINECONE_API_KEY"] = st.secrets["PINECONE_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = st.secrets["LANGCHAIN_API_KEY"]
PINECONE_INDEX = st.secrets["PINECONE_INDEX"]

co = cohere.Client(api_key=st.secrets["COHERE_API_KEY"])

embeddings = CohereEmbeddings(
    cohere_api_key=st.secrets["COHERE_API_KEY"], model="embed-multilingual-v3.0"
)
vectorstore = PineconeVectorStore(index_name=PINECONE_INDEX, embedding=embeddings)
retriever = vectorstore.as_retriever()


In [70]:
question = "Who is Nero"

documents = retriever.get_relevant_documents(question)

In [61]:
def format_documents(documents):
    formatted_documents = []
    for document in documents:
        formatted_documents.append({
            "title":document.metadata.get("sources"),
            "snippet": document.page_content,
        })
    return formatted_documents

In [71]:
formatted_documents = format_documents(documents)
rerank_documents = [doc['snippet'] for doc in formatted_documents]

results = co.rerank(model="rerank-multilingual-v3.0", query=question, documents=rerank_documents, top_n=3, return_documents=True)

INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/rerank "HTTP/1.1 200 OK"


In [76]:
print(results.results[0])

document=RerankResponseResultsItemDocument(text='Fig. 1. Prototype of the button interaction robot.\n   Cubetto, by Primotoys, is a cube-shaped robot, made with\nwooden boards that can be assembled and programmed on\nthe robot itself. This product also has a programmable table\nand directional blocks for planning the robot’s movement on a\nplayful board by allocating the directional blocks in sequential\ncommand lines on the programmable table [6], [7]. The LEGO\ngroup developed the LEGO-WeDo robot with a set of LEGO\nparts and geared motors that can be assembled to build the\nrobot and program it intuitively via USB connection to a\ncomputer with Scratch or Lego WeDo programs. From Meet\nEdison, the Edison Robot is a unicycle-type educational robot\nthat can be easily expandable using LEGO parts and can be\nprogrammed on an external computer using Scratch or Python.\n   Bee-Bot is a bee-like robot designed to be a friendly toy\nproducing sounds and lights, making it an attractive devi

In [55]:
preamble_template = '''

## Task & Context
----> TELL THE MODEL WHO IT IS AND WHAT IT DOES  <----

## Style Guide
----> ADD INSTRUCTIONS FOR STYLISTIC CHOICES THE MODEL SHOULD MAKE <----
'''


question = "O que é interação humana-robo?"

documents = retriever.get_relevant_documents("Who is more popular: Nsync or Backstreet Boys?")

for event in co.chat_stream(
    model="command-r-plus",
    message=question,
    preamble=preamble_template,
    documents=format_documents(documents)
):
    print(event)


INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"


generation_id='60ac326a-b52b-4d4a-b94f-dd72412c9da9' event_type='stream-start' is_finished=False
search_results=None documents=[{'id': 'doc_0', 'snippet': '9160', 'title': 'https://drive.google.com/file/d/1Ab0sLH_mAU_6eRs4YJNoR6h8sCZkxri-'}, {'id': 'doc_1', 'snippet': 'performs well for roll and pitch angles up to approximately\n30◦, corresponding to linear velocities of approximately\n\n\n\n\n\n1.5–2 m/s [48]. Upon such considerations, one can consider\n\n\n\n\n\n\n\n\n\n\n\n\nsin (θ)         ≈        θ,      sin (φ)           ≈       φ,       cos (θ)           ≈       cos (φ)            ≈\n1,    and      u1     ≈     mg,      and the translational dynamics in Eq. 3\ncan be written as\n    =(c ψ Kθ uθ              + sψ Kφ uφ) g − d1x\n\nx¨                                                             m ˙\n    =(s ψ Kθ uθ − cψ Kφ uφ) g − d2y\n\ny¨                                                             m ˙                                             (4)\n         K˙ u˙zz      − g − d

In [86]:
import os

import cohere
import streamlit as st
from langchain_cohere import CohereEmbeddings
from langchain_pinecone import PineconeVectorStore

os.environ["PINECONE_API_KEY"] = st.secrets["PINECONE_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = st.secrets["LANGCHAIN_API_KEY"]
PINECONE_INDEX = st.secrets["PINECONE_INDEX"]

co = cohere.Client(api_key=st.secrets["COHERE_API_KEY"])

embeddings = CohereEmbeddings(
    cohere_api_key=st.secrets["COHERE_API_KEY"], model="embed-multilingual-v3.0"
)
vectorstore = PineconeVectorStore(index_name=PINECONE_INDEX, embedding=embeddings)
retriever = vectorstore.as_retriever()


def format_documents(question):
    formatted_documents = []
    documents = retriever.get_relevant_documents(question)
    for document in documents:
        formatted_documents.append(
            {
                "title": f'[{document.metadata.get("article_title")}]{(document.metadata.get("sources"))}',
                "snippet": document.page_content,
            }
        )
    return formatted_documents


preamble_template = """

## Task & Context
Você é AuroRa, assistente virtual do Nero (Núcleo de Especialização em Róbótica). Seu papel é responder às perguntas usando partes de artigos científicos recuperados.
Geralmente os artigos estão em ingês, mas sua responsta deve ser em português a não ser que a pergunta do usuário esteja em inglês.
Se você não souber a resposta ou os artigos fornecidos não estão relacionados com a pergunta do usuário, apenas diga que não sabe. Use no máximo três frases e mantenha a resposta concisa. 

## Style Guide
Você deve ter um tom gentil e prestativo e usar linguagem acadêmica.
"""


def chat_answer(question):
    citations = []
    for event in co.chat_stream(
        model="command-r-plus",
        message=question,
        preamble=preamble_template,
        documents=format_documents(question),
    ):  
        print(event)
        print(event.event_type)
        print("*-*"*20)
        if event.event_type == "text-generation":
            ...
        
        if event.event_type == "citation-generation":
            citations.append(event.citations[0].document_ids)

        if event.event_type == "stream-end":
            citations = tuple(sum(citations, []))
            sources = []
            documents = event.response.documents
            for citation in citations:
                for document in documents:
                    if document['id'] == citation:
                        sources.append(document["title"])
            break
           

chat_answer("O que é interação humano robo?")


INFO:httpx:HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"


generation_id='771dc8be-515b-452d-8253-b80a0ff624d3' event_type='stream-start' is_finished=False
stream-start
*-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-*
search_results=None documents=[{'id': 'doc_0', 'snippet': 'Human-robot interaction can be defined as “a field of study\ndedicated to understand, design, and evaluate robotic systems\nfor use by or with humans” [1]. The interaction can occur in\ntwo different ways: remote and proximate. They are largely\ninfluenced by the proximity of the human and the robot.\nWorks like [2–5] bring good examples of how remote com-\nmunication can be exploited in several applications, such as\n\n\n\x02    Kevin Braathen de Carvalho\n     kevin.carvalho@ufv.br\n     Daniel Khede Dourado Villa\n     danielkdv@gmail.com\n\n     M´ario Sarcinelli-Filho\n     mario.sarcinelli@ufes.br\n     Alexandre Santos Brand˜ao\n     alexandre.brandao@ufv.br\n\n1    Graduate Program on Computer Science, Federal University\n     of Vic¸osa, 36570-900, Vi